In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Prepare Data

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    """Plot the series"""
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(False)

def trend(time, slope=0):
    """A trend over time"""
    return slope * time

def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.1,
                    np.cos(season_time * 6 * np.pi),
                    2 / np.exp(9 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    """Adds noise to the series"""
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

__Prepare time-series data__

In [ ]:
def generate_time_series():
    """ Creates timestamps and values of the time series """

    time = np.arange(4 * 365 + 1, dtype="float32")

    # Initial series is just a straight line with a y-intercept
    y_intercept = 10
    slope = 0.005
    series = trend(time, slope) + y_intercept

    # Adding seasonality
    amplitude = 50
    series += seasonality(time, period=365, amplitude=amplitude)

    # Adding some noise
    noise_level = 3
    series += noise(time, noise_level, seed=51)

    return time, series

In [ ]:
TIME, SERIES = generate_time_series()

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(TIME, SERIES)
plt.show()

__Split dataset__

In [ ]:
SPLIT_TIME = 1100

# Get the train set
time_train = TIME[:SPLIT_TIME]
series_train = SERIES[:SPLIT_TIME]

# Get the validation set
time_valid = TIME[SPLIT_TIME:]
series_valid = SERIES[SPLIT_TIME:]

__Generate dataset__

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """Generates dataset windows
    :param series (array of float) - contains the values of the time series
    :param window_size (int) - the number of time steps to include in the feature
    :param batch_size (int) - the batch size
    :param shuffle_buffer(int) - buffer size to use for the shuffle method
    :return dataset (TF Dataset) - TF Dataset containing time windows
    """

    # Add an axis for the feature dimension of RNN layers
    series = tf.expand_dims(series, axis=-1)
    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(series)
    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    # Create tuples with features and labels
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    # Shuffle the windows
    dataset = dataset.shuffle(shuffle_buffer)
    # Create batches of windows
    dataset = dataset.batch(batch_size)
    # Optimize the dataset for training
    dataset = dataset.cache().prefetch(1)

    return dataset

In [ ]:
WINDOW_SIZE = 20
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

# Generate dataset from given time-series data
dataset = windowed_dataset(series_train, WINDOW_SIZE, BATCH_SIZE, SHUFFLE_BUFFER_SIZE)

In [ ]:
for window in dataset.take(1):
  print(f'Shape of feature: {window[0].shape}')
  print(f'Shape of label: {window[1].shape}')

# Build And Compile Model

Steps:
* Create uncompiled model
* Compile and fit model with learning rate scheduler
* Identify the best learning rate
* Compile model with fixed learning rate

In [ ]:
def create_uncompiled_model():
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(WINDOW_SIZE, 1)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(1),
        tf.keras.layers.Lambda(lambda x: x * 100.0)
    ])
    return model

In [ ]:
def adjust_learning_rate(dataset, model):
    # Create scheduler callback
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-6 * 10**(epoch / 20))
    # Create optimizer without specifying learning rate
    optimizer = tf.keras.optimizers.SGD(momentum=0.9)
    # Compile the model passing
    model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
    # Fir the model
    history = model.fit(dataset, epochs=100, callbacks=[lr_schedule])
    return history

__Adjust learning rata__

In [ ]:
tf.keras.backend.clear_session()

# Create uncompiled model
uncompiled_model = create_uncompiled_model()
# Compile and fit model with learning rate scheduler
lr_history = adjust_learning_rate(dataset, uncompiled_model)

In [ ]:
# Plot the loss for every learning rates
plt.semilogx(lr_history.history["learning_rate"], lr_history.history["loss"])
_ = plt.axis((1e-6, 1, 0, 30))

In [ ]:
LEARNING_RATE=2e-6

__Create model__

In [ ]:
def create_model(learning_rate):
    # Create uncompiled model
    model = create_uncompiled_model()
    # Compile model with fixed learning rate
    model.compile(loss="mse",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9),
                  metrics=["mae"])
    return model

In [ ]:
tf.keras.backend.clear_session()

# Create final model with fixed learning rate
model = create_model(learning_rate=LEARNING_RATE)
# Fir final model
history = model.fit(dataset, epochs=50)

In [ ]:
loss = history.history["loss"]
epochs = len(loss)

plt.plot(range(len(loss)), loss, 'r', label='Training loss')
plt.title('Training loss')
plt.legend(loc=0)
plt.show()

__Evaluate model__

In [ ]:
def compute_metrics(true_series, forecast):
    """Computes MSE and MAE metrics for the forecast"""
    mse = tf.keras.losses.MSE(true_series, forecast)
    mae = tf.keras.losses.MAE(true_series, forecast)
    return mse, mae

In [ ]:
def generate_forecast(model, series, window_size):
    """Generates a forecast using your trained model"""
    forecast = []
    for time in range(SPLIT_TIME, len(series)):
        pred = model.predict(series[time-window_size:time][np.newaxis])
        forecast.append(pred[0][0])
    return forecast

In [ ]:
rnn_forecast = generate_forecast(model, SERIES, WINDOW_SIZE)

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, series_valid)
plot_series(time_valid, rnn_forecast)

In [ ]:
mse, mae = compute_metrics(series_valid, rnn_forecast)
print(f"mse: {mse:.2f}, mae: {mae:.2f} for forecast")